In [136]:
#export
from zero.imports import *
from zero.core import *

### Residual Network

In [130]:
#export
class PartialResStage(nn.Module):
    """
    Stage in a residual network, usually the units in a residual network are divided into
    stages according to feature (image) resolution.
    
    Parameters:
    -----------
    ni : number of input channels of the stage, 本stage的入channel数
    no : number of output channels of the stage, 本stage的出channel数
    nh : number of hidden channels of basic units in the stage, 内部channel数
    nu : number of basic units in the stage, unit数
    stride : stride size of conv op in First unit
    Unit : class of the basic unit, Unit class has calling format:
        Unit(ni:int, no:int, nh:int, stride:int=1, **kwargs)
    
    """
    def __init__(self, ni:int, no:int, nh:int, nu:int, stride:int, Unit:nn.Module,
                 a:int=1, **kwargs):
        super(PartialResStage, self).__init__()
        assert a < nu - 2
        self.a, self.nu = a, nu
        print(self.a, self.nu)
        # the first unit, stride size determine if downsample or not
        self.unit0 = Unit(ni, no, nh, stride=stride, **kwargs) 
        self.idmapping0 = IdentityMapping(ni, no, stride=stride) 
        units = []
        for i in range(nu - 1):
            units += [Unit(no, no, nh, stride=1, **kwargs)] #resnet_bottleneck
        self.units = nn.ModuleList(units)
            
    def forward(self, x):
        x_next = self.unit0(x) + self.idmapping0(x)
        x_pre = torch.zeros_like(x_next)
        x_sum = torch.zeros_like(x_next)
        for i, unit in enumerate(self.units):
            x_sum += x_next
            if i < self.a:
                x = x_next + x_pre
                x_pre = x
            elif i < self.nu - 2:
                x = x_next + x_pre
            else:
                x = x_next + x_sum
            x_next = unit(x)
        
        return x_next
        

In [5]:
#export
class ResStage(nn.Module):
    """
    Stage in a residual network, usually the units in a residual network are divided into
    stages according to feature (image) resolution.
    
    Parameters:
    -----------
    ni : number of input channels of the stage, 本stage的入channel数
    no : number of output channels of the stage, 本stage的出channel数
    nh : number of hidden channels of basic units in the stage, 内部channel数
    nu : number of basic units in the stage, unit数
    stride : stride size of conv op in First unit
    Unit : class of the basic unit, Unit class has calling format:
        Unit(ni:int, no:int, nh:int, stride:int=1, **kwargs)
    
    """
    def __init__(self, ni:int, no:int, nh:int, nu:int, stride:int, Unit:nn.Module, **kwargs):
        super(ResStage, self).__init__()
        # the first unit, stride size determine if downsample or not
        self.unit0 = Unit(ni, no, nh, stride=stride, **kwargs) 
        self.idmapping0 = IdentityMapping(ni, no, stride=stride) 
        units = []
        for i in range(nu - 1):
            units += [Unit(no, no, nh, stride=1, **kwargs)] #resnet_bottleneck
        self.units = nn.ModuleList(units)
            
    def forward(self, x):
        x = self.unit0(x) + self.idmapping0(x)
        for i, unit in enumerate(self.units):
            x = unit(x) + x
        return x
        

In [6]:
#export
def resnet_stem(ni:int=3, no:int=64):
    """Stem stage in resnet
    """
    return nn.Sequential(*[*conv_bn_relu(ni, 32, stride=2).children()],  #downsample
                         *[*conv_bn_relu(32, 32, stride=1).children()],
                         *[*conv_bn_relu(32, no, stride=1).children()],
                         nn.MaxPool2d(kernel_size=3, stride=2, padding=1)  #downsample
                        )

In [7]:
#export
class ResNet(nn.Sequential):
    """
    Residual Network
    
    Parameters:
    -----------
    nhs : number of hidden channels for all stages
    nos : number of output channels for all stages
    nus : number of units of all stages
    strides : stride sizes of all stages
    Stem : class of the stem layer, Stem class should has calling format:
        Stem(ni:int, no:int)
    Stage : class of the stages, Stage class has calling format:
        Stage(ni:int, no:int, nh:int, nu:int, stride:int, Unit:nn.Module, **kwargs)
    Unit : class of the basic units
    c_in : number of input channels of the whole network
    c_out : number of output channels (features) of the whole network
    kwargs : additional args to Unit class
    """
    def __init__(self, nhs, nos, nus, strides, Stem:nn.Module, Stage:nn.Module, Unit:nn.Module,
                 c_in:int=3, c_out:int=1000, **kwargs):
        super(ResNet, self).__init__()
        stem = Stem(c_in, nhs[0])
        stages = []
        ni = nhs[0]
        for i in range(len(nhs)):
            stages += [Stage(ni, nos[i], nhs[i], nus[i], strides[i], Unit, **kwargs)]
            ni = nos[i]
        classifier = Classifier(nos[-1], c_out)
        super().__init__(
            stem,
            *stages,
            classifier
        )
        init_cnn(self)
        
        
def resnet50(c_in:int=3, c_out:int=1000):
    return ResNet(nhs = [64, 128, 256, 512], nos = [256, 512, 1024, 2048],
                  nus = [3,4,6,3], strides = [1,2,2,2], Stem = resnet_stem, Stage = ResStage,
                  Unit = resnet_bottleneck,
                  c_in=c_in, c_out=c_out)

def resnet101(c_in:int=3, c_out:int=1000):
    return ResNet(nhs = [64, 128, 256, 512], nos = [256, 512, 1024, 2048],
                  nus = [3,4,23,3], strides = [1,2,2,2], Stem = resnet_stem, Stage = ResStage, 
                  Unit = resnet_bottleneck,
                  c_in=c_in, c_out=c_out)

In [8]:
model = resnet50()

In [9]:
num_params(model)

tensor(25576264)

### our folded stage classes

In [11]:
#export
class DualStage(nn.Module):
    def __init__(self, ni:int, no:int, nh:int, nu:int, stride:int, Unit:nn.Module, **kwargs):
        assert nu >= 2
        super(DualStage, self).__init__()
        # the first unit, stride size determine if downsample or not
        self.unit0 = Unit(ni, no, nh, stride=stride, **kwargs) 
        self.idmapping0 = IdentityMapping(ni, no, stride=stride) 
        self.unit1 = Unit(no, no, nh, stride=1, **kwargs) 
        units = []
        for i in range(nu - 2):
            units += [Unit(no, no, nh, stride=1, **kwargs)] #resnet_bottleneck
        self.units = nn.ModuleList(units)
            
    def forward(self, x):
        x1 = self.unit0(x) + self.idmapping0(x)
        x2 = self.unit1(x1) + x1
        for i, unit in enumerate(self.units):
            if i%2 == 0:
                x1 = unit(x2) + x1
            elif i%2 == 1:
                x2 = unit(x1) + x2
        return x2 if i%2 == 1 else x1
            

In [12]:
#export
class TripleStage(nn.Module):
    """
    Stage in a residual network, usually the units in a residual network are divided into
    stages according to feature (image) resolution.
    
    Parameters:
    -----------
    ni : number of input channels of the stage, 本stage的入channel数
    no : number of output channels of the stage, 本stage的出channel数
    nh : number of hidden channels of basic units in the stage, 内部channel数
    nu : number of basic units in the stage, unit数
    stride : stride size of conv op in First unit
    Unit : class of the basic unit, Unit class has calling format:
        Unit(ni:int, no:int, nh:int, stride:int=1, **kwargs)
    
    """
    def __init__(self, ni:int, no:int, nh:int, nu:int, stride:int, Unit:nn.Module, **kwargs):
        assert nu >= 4
        super(TripleStage, self).__init__()
        # the first unit, stride size determine if downsample or not
        self.unit0 = Unit(ni, no, nh, stride=stride, **kwargs) 
        self.idmapping0 = IdentityMapping(ni, no, stride=stride) 
        self.unit1 = Unit(no, no, nh, stride=1, **kwargs) 
        self.unit2 = Unit(no, no, nh, stride=1, **kwargs) 
        units = []
        for i in range(nu - 3):
            units += [Unit(no, no, nh, stride=1, **kwargs)] #resnet_bottleneck
        self.units = nn.ModuleList(units)
            
    def forward(self, x):
        x1 = self.unit0(x) + self.idmapping0(x)
        x2 = self.unit1(x1) + x1
        x3 = self.unit2(x2) + x2
        for i, unit in enumerate(self.units):
            if i%4 == 0:
                x2 = unit(x3) + x2
            elif i%4 == 1:
                x1 = unit(x2) + x1
            elif i%4 == 2:
                x2 = unit(x1) + x2
            elif i%4 == 3:
                x3 = unit(x2) + x3
        return x2 if i%4 == 0 else x1 if i%4 == 1 else x2 if i%4 == 2 else x3 #if i%4 == 3
        

In [13]:
#export
class QuadStage(nn.Module):
    """
    Stage in a residual network, usually the units in a residual network are divided into
    stages according to feature (image) resolution.
    
    Parameters:
    -----------
    ni : number of input channels of the stage, 本stage的入channel数
    no : number of output channels of the stage, 本stage的出channel数
    nh : number of hidden channels of basic units in the stage, 内部channel数
    nu : number of basic units in the stage, unit数
    stride : stride size of conv op in First unit
    Unit : class of the basic unit, Unit class has calling format:
        Unit(ni:int, no:int, nh:int, stride:int=1, **kwargs)
    
    """
    def __init__(self, ni:int, no:int, nh:int, nu:int, stride:int, Unit:nn.Module, **kwargs):
        assert nu >= 5
        super(QuadStage, self).__init__()
        # the first unit, stride size determine if downsample or not
        self.unit0 = Unit(ni, no, nh, stride=stride, **kwargs) 
        self.idmapping0 = IdentityMapping(ni, no, stride=stride) 
        self.unit1 = Unit(no, no, nh, stride=1, **kwargs) 
        self.unit2 = Unit(no, no, nh, stride=1, **kwargs) 
        self.unit3 = Unit(no, no, nh, stride=1, **kwargs) 
        units = []
        for i in range(nu - 4):
            units += [Unit(no, no, nh, stride=1, **kwargs)] #resnet_bottleneck
        self.units = nn.ModuleList(units)
            
    def forward(self, x):
        x1 = self.unit0(x) + self.idmapping0(x)
        x2 = self.unit1(x1) + x1
        x3 = self.unit2(x2) + x2
        x4 = self.unit3(x3) + x3
        for i, unit in enumerate(self.units):
            if i % 6 == 0:
                x3 = unit(x4) + x3
            elif i % 6 == 1:
                x2 = unit(x3) + x2
            elif i % 6 == 2:
                x1 = unit(x2) + x1
            elif i % 6 == 3:
                x2 = unit(x1) + x2
            elif i % 6 == 4:
                x3 = unit(x2) + x3
            elif i % 6 == 5:
                x4 = unit(x3) + x4
        return x3 if i%6 == 0 or i%6 ==4 else x2 if i%6 == 1 or i%6 == 3 else x1 if i%6 == 2 else x4 # if i%6 == 5        

In [14]:
#export
class FiveStage(nn.Module):
    """
    Stage in a residual network, usually the units in a residual network are divided into
    stages according to feature (image) resolution.
    
    Parameters:
    -----------
    ni : number of input channels of the stage, 本stage的入channel数
    no : number of output channels of the stage, 本stage的出channel数
    nh : number of hidden channels of basic units in the stage, 内部channel数
    nu : number of basic units in the stage, unit数
    stride : stride size of conv op in First unit
    Unit : class of the basic unit, Unit class has calling format:
        Unit(ni:int, no:int, nh:int, stride:int=1, **kwargs)
    
    """
    def __init__(self, ni:int, no:int, nh:int, nu:int, stride:int, Unit:nn.Module, **kwargs):
        assert nu >= 6
        super(FiveStage, self).__init__()
        # the first unit, stride size determine if downsample or not
        self.unit0 = Unit(ni, no, nh, stride=stride, **kwargs) 
        self.idmapping0 = IdentityMapping(ni, no, stride=stride) 
        self.unit1 = Unit(no, no, nh, stride=1, **kwargs) 
        self.unit2 = Unit(no, no, nh, stride=1, **kwargs) 
        self.unit3 = Unit(no, no, nh, stride=1, **kwargs) 
        self.unit4 = Unit(no, no, nh, stride=1, **kwargs) 
        units = []
        for i in range(nu - 5):
            units += [Unit(no, no, nh, stride=1, **kwargs)] #resnet_bottleneck
        self.units = nn.ModuleList(units)
            
    def forward(self, x):
        x1 = self.unit0(x) + self.idmapping0(x)
        x2 = self.unit1(x1) + x1
        x3 = self.unit2(x2) + x2
        x4 = self.unit3(x3) + x3
        x5 = self.unit4(x4) + x4
        for i, unit in enumerate(self.units):
            if i % 8 == 0:
                x4 = unit(x5) + x4
            elif i % 8 == 1:
                x3 = unit(x4) + x3
            elif i % 8 == 2:
                x2 = unit(x3) + x2
            elif i % 8 == 3:
                x1 = unit(x2) + x1
            elif i % 8 == 4:
                x2 = unit(x1) + x2
            elif i % 8 == 5:
                x3 = unit(x2) + x3
            elif i % 8 == 6:
                x4 = unit(x3) + x4
            elif i % 8 == 7:
                x5 = unit(x4) + x5
        return x4 if i%8 == 0 or i%8 == 6 else x3 if i%8 == 1 or i%8 == 5 else x2 if i%8 == 2 or i%8 == 4 else x1 if i%8 == 3 else x5 # if i%8 == 7

In [45]:
#export
def folded_resnet(Stem, Stage, Unit, ni:int=32, num_stages:int=4, num_units:int=6, exp:int=2,
                  bottle_scale:int=1, first_downsample:bool=True, c_in:int=3, c_out:int=10, **kwargs):
    """
    A folded residual network.
    
    Parameters:
    -----------
    Stem : class of the stem layer.
    Stage : class of the stages.
    Unit : class of the basic blocks.
    ni  :  number of input channels of the first stage 
            equal to number of output channels of the stem layer.
    num_stages : number of stages.
    num_units : number of units per stage.
    exp : expansion coefficient for number of channels increasing with stages.
    bottle_scale : bottleneck coefficient.
    first_downsample : does downsample at the first stage.
    c_in : number of input channels of the stem layer.
    c_out : 
    """
    nhs = [ni * exp ** i for i in range(num_stages)] # [ni] + [exp*ni] + [exp*ni] + [exp*ni]
    nos = [nh*bottle_scale for nh in nhs]
    nus = [num_units] * num_stages  # all stages have the same number of units
    strides = [1 if i==0 and not first_downsample else 2 for i in range(num_stages)]
    print(nhs, nos, nus, strides)
    return ResNet(nhs = nhs, nos = nos,
                  nus = nus, strides = strides, Stem = Stem, Stage = Stage,
                  Unit = Unit,
                  c_in=c_in, c_out=c_out, **kwargs)


In [131]:
model = folded_resnet(Stem = conv_bn, Stage = PartialResStage, Unit = xception, ni=64,
                      num_stages=5, num_units=10, exp=1, first_downsample=False, zero_bn=True, a = 2)

[64, 64, 64, 64, 64] [64, 64, 64, 64, 64] [10, 10, 10, 10, 10] [1, 2, 2, 2, 2]
2 10
2 10
2 10
2 10
2 10


In [132]:
num_params(model)

tensor(242506)

In [133]:
x = torch.randn(2,3,64,64)

In [134]:
with torch.autograd.set_detect_anomaly(True):
    out = model(x)
    out.mean().backward()

### Export: Before Export, First Remove `zero` in `from zero.bla import *`

In [139]:
!python notebook2script.py resnet.ipynb

Converted resnet.ipynb to zero/resnet.py
